In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:


  %reload_ext autoreload


In [2]:
import os
import sys
from multiprocessing import Pool
from tqdm.auto import tqdm

In [3]:
project_root = os.path.abspath(
    os.path.join("C:/Users/jllgo/OneDrive/Documentos/UFMG/MSc & MSI 2/MatchSegmentation")
)
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from src.pass_networks.process_games import process_game
from src.pass_networks.process_games import load_game
from src.pass_networks.process_games import save_game

from src.pass_networks.process_sequences import process_sequences

In [5]:
RAW_DATA_PATH = '../data/raw/PL-22-23'
INTERMEDIATE_DATA_PATH = '../data/intermediate/PL-22-23'
PROCESSED_DATA_PATH = "../data/processed/PL-22-23"

# 1. Get Data

In [6]:
games = os.listdir(RAW_DATA_PATH)
game_ids = [game.split('.')[0] for game in games][:5]
print(f"Games to process: {len(game_ids)}")

Games to process: 5


In [7]:
pre_processed_games = os.listdir(INTERMEDIATE_DATA_PATH)
pre_processed_game_ids = [game.split('.')[0] for game in pre_processed_games]
print(f"Games already pre-processed: {len(pre_processed_game_ids)}")

Games already pre-processed: 0


In [8]:
game_ids = [game_id for game_id in game_ids if game_id not in pre_processed_game_ids]
print(f"Games to process: {len(game_ids)}")

Games to process: 5


In [9]:
frames = []

# Check if the game has already been pre-processed
if len(pre_processed_game_ids) > 0:
    tasks = [(INTERMEDIATE_DATA_PATH,game_id) for game_id in pre_processed_game_ids]

    for task in tqdm(tasks, desc="Loading Pre-Processed Games", total=len(tasks)):
        _,metadata_df, players_df, events_df = load_game(task)
        frames.append((metadata_df, players_df, events_df))

else:

    tasks = [(RAW_DATA_PATH,game_id) for game_id in game_ids]

    for task in tqdm(tasks, desc="Processing Games", total=len(tasks)):
        _,metadata_df, players_df, events_df = process_game(task)
        frames.append((metadata_df, players_df, events_df))
        save_game(metadata_df,players_df,events_df,INTERMEDIATE_DATA_PATH,task[1])

Processing Games:   0%|          | 0/5 [00:00<?, ?it/s]

Loading frames from match 4436: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/170699 [00:00<?, ? frames/s]


100%|██████████| 2/2 [00:02<00:00,  1.02s/it]

100%|██████████| 51/51 [00:02<00:00, 20.00it/s]


Loading frames from match 4437: 0 frames [00:00, ? frames/s]

...

Validating frames:   0%|          | 0/173901 [00:00<?, ? frames/s]


100%|██████████| 2/2 [00:05<00:00,  2.76s/it]

100%|██████████| 49/49 [00:08<00:00,  5.52it/s]


Loading frames from match 4440: 0 frames [00:00, ? frames/s]

Validating frames:   0%|          | 0/170271 [00:00<?, ? frames/s]


100%|██████████| 2/2 [00:07<00:00,  3.57s/it]

100%|██████████| 50/50 [00:09<00:00,  5.54it/s]


# 2. Get Graphs

In [27]:
frames[0][-1].columns

Index(['match_id', 'event_id', 'possession_id', 'possession_type', 'player_id',
       'receiver', 'outcome', 'carry_type'],
      dtype='object')

In [28]:
process_sequences(frames[0])

Preparing sequences:   0%|          | 0/300 [00:00<?, ?it/s]

Processing data:   0%|          | 0/300 [00:00<?, ?it/s]

PicklingError: Can't pickle <function sequence_to_graph at 0x000002EAABE5C4A0>: it's not the same object as src.pass_networks.pass_network.sequence_to_graph